# libCEED for Python examples

This is a tutorial to illustrate the main feautures of the Python interface for [libCEED](https://github.com/CEED/libCEED/), the low-level API library for efficient high-order discretization methods developed by the co-design [Center for Efficient Exascale Discretizations](https://ceed.exascaleproject.org/) (CEED) of the [Exascale Computing Project](https://www.exascaleproject.org/) (ECP).

While libCEED's focus is on high-order finite/spectral element method implementations, the approach is mostly algebraic and thus applicable to other discretizations in factored form, as explained in the [user manual](https://libceed.org/).

## Setting up libCEED for Python

Install libCEED for Python by running

In [ ]:
! python -m pip install libceed

## CeedBasis

Here we show some basic examples to illustrate the `libceed.Basis` class. In libCEED, a `libceed.Basis` defines the finite element basis and associated quadrature rule (see [the API documentation](https://libceed.org/en/latest/libCEEDapi.html#finite-element-operator-decomposition)).

First we declare some auxiliary functions needed in the following examples

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def eval(dim, x):
  result, center = 1, 0.1
  for d in range(dim):
    result *= np.tanh(x[d] - center)
    center += 0.1
  return result

def feval(x_1, x_2):
  return x_1*x_1 + x_2*x_2 + x_1*x_2 + 1

def dfeval(x_1, x_2):
  return 2*x_1 + x_2

## $H^1$ Lagrange bases in 1D

The Lagrange interpolation nodes are at the Gauss-Lobatto points, so interpolation to Gauss-Lobatto quadrature points is the identity.

In [ ]:
import libceed

ceed = libceed.Ceed()

b = ceed.BasisTensorH1Lagrange(
    dim=1,   # topological dimension
    ncomp=1, # number of components
    P=4,     # number of basis functions (nodes) per dimension
    Q=4,     # number of quadrature points per dimension
    qmode=libceed.GAUSS_LOBATTO)
print(b)

Although a `libceed.Basis` is fully discrete, we can use the Lagrange construction to extend the basis to continuous functions by applying `EVAL_INTERP` to the identity.  This is the Vandermonde matrix of the continuous basis.

In [ ]:
P = b.get_num_nodes()
Q_viz = 50
basis_viz = ceed.BasisTensorH1Lagrange(1, 1, P, Q_viz, libceed.GAUSS_LOBATTO)

# Construct P "elements" with one node activated
I = ceed.Vector(P * P)
with I.array_write(P, P) as x:
    x[...] = np.eye(P)

basis_fns = ceed.Vector(P * Q_viz)
basis_viz.apply(4, libceed.EVAL_INTERP, I, basis_fns)

qpts_viz, _ = ceed.lobatto_quadrature(Q_viz)
with basis_fns.array_read(Q_viz, P) as B_array:
    plt.plot(qpts_viz, B_array)

# Mark tho Lobatto nodes
nodes, _ = ceed.lobatto_quadrature(P)
plt.plot(nodes, 0*nodes, 'ok');

In contrast, the Gauss quadrature points are not collocated, and thus all basis functions are generally nonzero at every quadrature point.

In [ ]:
b = ceed.BasisTensorH1Lagrange(1, 1, 4, 4, libceed.GAUSS)
print(b)

with basis_fns.array_read(Q_viz, P) as B_array:
    plt.plot(qpts_viz, B_array)
# Mark tho Gauss quadrature points
qpts, _ = ceed.gauss_quadrature(P)
plt.plot(qpts, 0*qpts, 'ok');

Although the underlying functions are not an intrinsic property of a `libceed.Basis` in libCEED, the sizes are.
Here, we create a 3D tensor product element with more quadrature points than Lagrange interpolation nodes.

In [ ]:
b = ceed.BasisTensorH1Lagrange(3, 1, 4, 5, libceed.GAUSS_LOBATTO)

p = b.get_num_nodes()
print('p =', p)

q = b.get_num_quadrature_points()
print('q =', q)

* In the following example, we demonstrate the application of an interpolatory basis in multiple dimensions

In [ ]:
for dim in range(1, 4):
  Q = 4
  Q_dim = Q**dim
  X_dim = 2**dim
  x = np.empty(X_dim*dim, dtype="float64")
  u_array = np.empty(Q_dim, dtype="float64")

  for d in range(dim):
    for i in range(X_dim):
      x[d*X_dim + i] = 1 if (i % (2**(dim-d))) // (2**(dim-d-1)) else -1

  X = ceed.Vector(X_dim*dim)
  X.set_array(x, cmode=libceed.USE_POINTER)
  X_q = ceed.Vector(Q_dim*dim)
  X_q.set_value(0)
  U = ceed.Vector(Q_dim)
  U.set_value(0)
  U_q = ceed.Vector(Q_dim)

  basis_x_lobatto = ceed.BasisTensorH1Lagrange(dim, dim, 2, Q, libceed.GAUSS_LOBATTO)
  basis_u_lobatto = ceed.BasisTensorH1Lagrange(dim, 1, Q, Q, libceed.GAUSS_LOBATTO)

  basis_x_lobatto.apply(1, libceed.EVAL_INTERP, X, X_q)

  with X_q.array_read() as x_array:
    for i in range(Q_dim):
      x = np.empty(dim, dtype="float64")
      for d in range(dim):
        x[d] = x_array[d*Q_dim + i]
      u_array[i] = eval(dim, x)

  U_q.set_array(u_array, cmode=libceed.USE_POINTER)

  # This operation is the identity because the quadrature is collocated
  basis_u_lobatto.T.apply(1, libceed.EVAL_INTERP, U_q, U)

  basis_x_gauss = ceed.BasisTensorH1Lagrange(dim, dim, 2, Q, libceed.GAUSS)
  basis_u_gauss = ceed.BasisTensorH1Lagrange(dim, 1, Q, Q, libceed.GAUSS)

  basis_x_gauss.apply(1, libceed.EVAL_INTERP, X, X_q)
  basis_u_gauss.apply(1, libceed.EVAL_INTERP, U, U_q)

  with X_q.array_read() as x_array, U_q.array_read() as u_array:
    if dim == 2:
        # Default ordering is contiguous in x direction, but
        # pyplot expects meshgrid convention, which is transposed.
        x, y = x_array.reshape(2, Q, Q).transpose(0, 2, 1)
        plt.scatter(x, y, c=np.array(u_array).reshape(Q, Q))
        plt.xlim(-1, 1)
        plt.ylim(-1, 1)
        plt.colorbar(label='u')

* In the following example, we demonstrate the application of the gradient of the shape functions in multiple dimensions

In [ ]:
for dim in range (1, 4):
  P, Q = 8, 10
  P_dim = P**dim
  Q_dim = Q**dim
  X_dim = 2**dim
  sum_1 = sum_2 = 0
  x_array = np.empty(X_dim*dim, dtype="float64")
  u_array = np.empty(P_dim, dtype="float64")

  for d in range(dim):
    for i in range(X_dim):
      x_array[d*X_dim + i] = 1 if (i % (2**(dim-d))) // (2**(dim-d-1)) else -1

  X = ceed.Vector(X_dim*dim)
  X.set_array(x_array, cmode=libceed.USE_POINTER)
  X_q = ceed.Vector(P_dim*dim)
  X_q.set_value(0)
  U = ceed.Vector(P_dim)
  U_q = ceed.Vector(Q_dim*dim)
  U_q.set_value(0)
  Ones = ceed.Vector(Q_dim*dim)
  Ones.set_value(1)
  G_transpose_ones = ceed.Vector(P_dim)
  G_transpose_ones.set_value(0)

  # Get function values at quadrature points
  basis_x_lobatto = ceed.BasisTensorH1Lagrange(dim, dim, 2, P, libceed.GAUSS_LOBATTO)
  basis_x_lobatto.apply(1, libceed.EVAL_INTERP, X, X_q)

  with X_q.array_read() as x_array:
    for i in range(P_dim):
      x = np.empty(dim, dtype="float64")
      for d in range(dim):
        x[d] = x_array[d*P_dim + i]
      u_array[i] = eval(dim, x)

  U.set_array(u_array, cmode=libceed.USE_POINTER)

  # Calculate G u at quadrature points, G' * 1 at dofs
  basis_u_gauss = ceed.BasisTensorH1Lagrange(dim, 1, P, Q, libceed.GAUSS)
  basis_u_gauss.apply(1, libceed.EVAL_GRAD, U, U_q)
  basis_u_gauss.T.apply(1, libceed.EVAL_GRAD, Ones, G_transpose_ones)

  # Check if 1' * G * u = u' * (G' * 1)
  with G_transpose_ones.array_read() as g_array, U_q.array_read() as uq_array:
    for i in range(P_dim):
      sum_1 += g_array[i]*u_array[i]
    for i in range(dim*Q_dim):
      sum_2 += uq_array[i]

  # Check that (1' * G * u - u' * (G' * 1)) is numerically zero
  print('1T * G * u - uT * (GT * 1) =', np.abs(sum_1 - sum_2))